In [ ]:
import pandas as pd

In [ ]:
SELECTED_CONCEPTS = [1, 4, 6, 7, 10, 14, 15, 20, 21, 23, 25, 29, 30, 35, 36, 38, 40, 44, 45,
                     50, 51, 53, 54, 56, 57, 59, 63, 64, 69, 70, 72, 75, 80, 84, 90, 91, 93,
                     99, 101, 106, 110, 111, 116, 117, 119, 125, 126, 131, 132, 134, 145, 149,
                     151, 152, 153, 157, 158, 163, 164, 168, 172, 178, 179, 181, 183, 187, 188,
                     193, 194, 196, 198, 202, 203, 208, 209, 211, 212, 213, 218, 220, 221, 225,
                     235, 236, 238, 239, 240, 242, 243, 244, 249, 253, 254, 259, 260, 262, 268,
                     274, 277, 283, 289, 292, 293, 294, 298, 299, 304, 305, 308, 309, 310, 311]

# Without shape/size
SELECTED_CONCEPTS_V2 = [1, 4, 6, 7, 10, 14, 15, 20, 21, 23, 25, 29, 30, 35, 36, 38, 40, 44, 45,
                        50, 51, 53, 54, 56, 57, 59, 63, 64, 69, 70, 72, 75, 80, 84, 90, 91, 93,
                        99, 101, 106, 110, 111, 116, 117, 119, 125, 126, 131, 132, 134, 145, 149,
                        151, 152, 153, 157, 158, 163, 164, 168, 172, 178, 179, 181, 183, 187, 188,
                        193, 194, 196, 198, 202, 203, 208, 209, 211, 212, 213, 236, 238, 239, 240,
                        242, 243, 244, 249, 253, 254, 259, 260, 262, 268, 274, 277, 283, 289, 292,
                        293, 294, 298, 299, 304, 305, 308, 309, 310, 311]

In [ ]:
attribute_df = pd.read_csv('datasets/CUB/attributes.txt', sep=' ', header=None, names=['attribute_id', 'attribute_name']).drop(columns=['attribute_id'])
attribute_df = attribute_df.iloc[SELECTED_CONCEPTS_V2]
fine_grained_parts = ['bill', 'upperpart', 'underpart', 'head', 'back', 'beak', 'belly', 'breast',
                      'crown', 'eye', 'forehead', 'leg', 'nape', 'tail', 'throat', 'wing', 'primary']
coarse_grained_parts = ['head', 'back', 'belly', 'breast', 'leg', 'tail', 'wing']
fine2coarse =  {
    'bill': 'head',
    'upperpart': 'breast',
    'underpart': 'belly',
    'head': 'head',
    'back': 'back',
    'beak': 'head',
    'belly': 'belly',
    'breast': 'breast',
    'crown': 'head',
    'eye': 'head',
    'forehead': 'head',
    'leg': 'leg',
    'nape': 'back',
    'tail': 'tail',
    'throat': 'breast',
    'wing': 'wing',
    'primary': 'breast'
}

In [ ]:
part_pattern = '|'.join(fine_grained_parts)
attribute_df['part_name'] = attribute_df['attribute_name'].str.extract(f'({part_pattern})')
attribute_df['part_name'] = attribute_df['part_name'].replace(fine2coarse)
attribute_df['part_id'] = attribute_df['part_name'].map(coarse_grained_parts.index)
attribute_df

In [ ]:
attribute_df['part_id'].to_numpy()

In [ ]:
import torch

In [ ]:
16 * 107

In [1]:
import os
import Augmentor
from torch.utils.data import DataLoader
import torchvision.transforms as T
from data.cub import CUBDatasetSimple

p = Augmentor.Pipeline()
p.rotate(probability=0.4, max_left_rotation=10, max_right_rotation=10)
p.shear(probability=0.4, max_shear_left=10, max_shear_right=10)
p.random_distortion(probability=0.4, grid_height=16, grid_width=16, magnitude=8)
p.skew(probability=0.4)
train_transforms = T.Compose([T.Resize(448), T.CenterCrop(448), p.torch_transform(), T.RandomHorizontalFlip(p=0.4), T.PILToTensor()])
test_transforms = T.Compose([T.Resize(448), T.CenterCrop(448), T.PILToTensor()])
dataset_train = CUBDatasetSimple(os.path.join('datasets', 'CUB'), split='train', transforms=train_transforms)
dataloader_train = DataLoader(dataset=dataset_train, batch_size=4, shuffle=True)
dataloader_train_iter = iter(dataloader_train)

In [ ]:
images, class_tgts, attr_tgts, all_class_attrs, attr_group_ids = next(dataloader_train_iter)
images.shape, class_tgts.shape, attr_tgts.shape, all_class_attrs.shape, attr_group_ids.shape

In [2]:
dataset_train.attr_class_map

tensor([[ 2.9197, 59.8540,  3.6496,  ...,  5.3571, 21.4286, 19.6429],
        [ 4.4118, 57.3529,  0.0000,  ...,  3.8095, 10.4762,  8.5714],
        [ 3.9735, 70.8609,  5.2980,  ...,  0.9346,  7.4766, 20.5607],
        ...,
        [36.7089,  0.6329, 43.6709,  ..., 44.2953, 18.7919, 18.1208],
        [ 2.0690,  0.0000, 88.2759,  ..., 34.4828, 33.1034, 20.0000],
        [ 8.2803,  0.0000, 82.8025,  ...,  3.0534, 10.6870, 29.7710]],
       dtype=torch.float64)